In [1]:
import pandas as pd
import numpy as np
import collections
import re
import nltk
import json
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk.tokenize import word_tokenize

In [4]:
import ssl
import certifi
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('stopwords')
stop=stopwords.words('english')
custom = ['?','(', ')', '.', '[', ']','!', '...',
';',"`","'",'"',',', "'s", "'ll", 'ca', "n't", "'m", "'re", "'ve"]
stop.extend(custom)

[nltk_data] Downloading package stopwords to /Users/adnan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
WNlemma = nltk.WordNetLemmatizer()
def cleaner(data):
    data=re.sub(r'\r',"",data)
    data=re.sub(r'\n'," ",data)
    data=re.sub(r' ( )+'," ",data)
    data=re.sub(r',',"",data)
    table=str.maketrans('', '', string.punctuation)
    data=data.translate(table)
    tokens = word_tokenize(data)
    new_docs=""
    for i in tokens:
        if i not in stop:
            new_docs+=WNlemma.lemmatize(i)+" "
    return new_docs[:len(new_docs)-1] 

In [4]:
f = open('lyrics.json')
data = json.load(f)

In [5]:
postionalIndex=dict()
docs_list=[]
docs=[]
for i in data:
    docs.append(i["lyrics"].lower())
    docs_list.append(cleaner(i["lyrics"].lower()))

In [6]:
def buildPostionalIndex():
    for j in range(len(docs)):
        temp=docs[j].lower().strip().split()
        for i in range(len(temp)):
            if temp[i] not in postionalIndex.keys():
                postionalIndex[temp[i]]=dict()
            if j not in postionalIndex[temp[i]].keys():   
                postionalIndex[temp[i]][j]=set()
            postionalIndex[temp[i]][j].add(i)

In [7]:
buildPostionalIndex()

In [8]:
postionalIndex

{'so,': {0: {0}, 12: {120, 203}, 53: {26, 27}, 58: {480, 483}, 88: {158}},
 'so': {0: {1},
  6: {181},
  8: {98},
  12: {109},
  13: {432, 599},
  16: {77},
  17: {7},
  21: {148},
  22: {42, 93, 217, 233},
  28: {6, 14, 30, 39, 55, 63, 79, 88, 99, 111, 119, 135, 146, 158},
  29: {150},
  34: {91},
  35: {102},
  40: {20},
  42: {98},
  46: {5},
  49: {105},
  51: {128, 134},
  58: {436, 481, 484},
  59: {5, 12, 47, 121, 128},
  61: {0, 2, 43, 45, 58, 60},
  64: {25, 51, 123},
  65: {164},
  68: {95},
  70: {98},
  71: {86},
  72: {93},
  73: {80, 101},
  74: {14, 50, 225},
  78: {178, 192, 380},
  79: {30, 34, 199},
  80: {79, 163},
  82: {32, 59, 161},
  83: {189},
  85: {84},
  87: {43, 134},
  91: {74},
  94: {133},
  95: {84},
  96: {110},
  97: {186},
  98: {91, 148},
  100: {52}},
 'you': {0: {2, 4, 15, 31, 33, 39, 61, 83, 114},
  1: {2,
   6,
   10,
   13,
   17,
   21,
   24,
   27,
   31,
   35,
   39,
   46,
   49,
   53,
   57,
   61,
   65,
   69,
   73},
  2: {86, 106, 11

In [9]:
def cmp(word,word2):
    res=[]
   
    for i in word.keys():
    
        for j in word2.keys():
            if i==j:
               
                res1=dict()
                for k in word[i]:
                    for l in word2[j]:
                        
                        if k+1==l:
                         
                            if i not in res1.keys():
                                res1[i]=set()
                            res1[i].add(l) 
                res.append(res1)
    return res   

In [10]:
def querySearch(sentance):
    temp=sentance.lower().strip().split()
    if temp[0] not in postionalIndex.keys():
        return []
    res=postionalIndex[temp[0]]
    for i in range(1,len(temp)):
        if temp[i] in postionalIndex.keys():
            if i==1:
                res=cmp(res,postionalIndex[temp[i]])
            else:
                res=cmp({ 'temp':j for j in res }["temp"],postionalIndex[temp[i]])
        else:
            res={}
    return res

In [11]:

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(docs)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df1 = pd.DataFrame(denselist, columns=feature_names)

In [12]:

df1.head()

,129,225,30,69,ability,about,above,acelera,acorns,across,...,youll,young,youngest,your,yours,yourself,yuppies,zion,zoo,único
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.039728,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.054263,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.057884,0.0,0.055931,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.028988,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_dic=df1.to_dict()

In [14]:
posting_list1=dict()
for key,value in df_dic.items():
    if key not in posting_list1.keys():
        posting_list1[key]=[]
    for key1,value1 in value.items():
        if value1>0.0:
            posting_list1[key].append(key1)

In [15]:

def search_rank2(word):
    if type(word)==str:
        if word not in df1.columns:
            return {}
        temp=df1[word].to_dict()
    else:
        temp=word
    res=dict()
    while len(temp)>0:
        v = list(temp.values())
        k = list(temp.keys())
        res[k[v.index(max(v))]]=max(v)
        del temp[k[v.index(max(v))]]
    return res 

In [16]:
t=querySearch("you jj")

In [17]:
soundexDic={"a":0,"i":0,"u":0,"e":0,"o":0,"h":0,"w":0,"y":0,"b":1,"f":1,"v":1,"p":1,"c":2,
            "g":2,"j":2,"k":2,"q":2,"s":2,"x":2,"z":2,"d":3,"t":3,"l":4,"m":5,"n":5,"r":6}

In [18]:
def wordSound(word):
    res=""
    word=word.lower().strip()
    for i in range(len(word)):
        if i==0:
            res+=word[i]
        elif word[i] in soundexDic.keys():
            res+=str(soundexDic[word[i]])
    if(len(res)<4):
        for i in range((4-len(res))):
            res+="0"
    return res[:4]   

In [19]:
def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)

    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])

    return res

In [20]:
def literalSearch(sentence):
    t=querySearch(sentence)
    k=[]
    ress=[]
    if type(t)==list:
        for i in range(len(t)):
            if t[i]!={}:
                k.append(i)
    else:
        for i in t.keys():
            if t[i]!={}:
                k.append(i)
    for key in k:        
        ress.append(data[key]["artist"]+": "+data[key]["title"])    
    return ress

In [21]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(docs_list)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [22]:
df.head()

,1294,225,30,69,ability,acelera,acorn,across,act,acting,...,youd,youll,young,youngest,youre,youve,yuppie,zion,zoo,único
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.046990,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0688,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.048204,0.0,0.0,0.0,0.0,0.0


In [23]:
df_dic=df.to_dict()

In [24]:
posting_list=dict()
for key,value in df_dic.items():
    if key not in posting_list.keys():
        posting_list[key]=[]
    for key1,value1 in value.items():
        if value1>0.0:
            posting_list[key].append(key1)

In [25]:
posting_list

{'1294': [5],
 '225': [91],
 '30': [10],
 '69': [75],
 'ability': [51],
 'acelera': [11],
 'acorn': [23],
 'across': [5, 34],
 'act': [78],
 'acting': [98],
 'adam': [88],
 'addict': [58],
 'adding': [4],
 'adios': [87],
 'aflowin': [47],
 'afraid': [51],
 'afro': [83],
 'age': [71],
 'agree': [12],
 'ah': [7, 9, 11, 19, 24, 38],
 'aha': [78],
 'ahah': [9],
 'ahead': [84],
 'ahh': [86],
 'ahora': [10],
 'aint': [23, 71, 74, 75, 78, 82, 89, 93, 96],
 'air': [0, 5, 55],
 'airbag': [57],
 'aircrash': [48],
 'aisle': [39],
 'al': [9],
 'alabama': [89],
 'alarm': [40, 49],
 'album': [35],
 'alien': [55],
 'alive': [71],
 'alone': [5, 12, 36, 54, 84],
 'along': [6, 30, 82, 98],
 'already': [76],
 'alright': [14, 55, 59, 96],
 'also': [51],
 'although': [74],
 'altimeter': [5],
 'altitude': [6],
 'always': [13, 16, 18, 32, 35, 39, 49, 53, 59, 65, 69, 80, 82, 87, 88],
 'amazed': [57],
 'ambition': [6, 56],
 'american': [4],
 'amigo': [10],
 'amo': [10],
 'amore': [11, 13, 33],
 'amsterdam': [2

In [26]:

def search_rank1(word):
    word=WNlemma.lemmatize(word)
    if word not in df.columns:
        return {}
    temp=df[word].to_dict()
    res=dict()
    while len(temp)>0:
        v = list(temp.values())
        k = list(temp.keys())
        res[k[v.index(max(v))]]=max(v)
        del temp[k[v.index(max(v))]]
    return res    

In [27]:
search_rank1("always")

{65: 0.29024033667328614,
 49: 0.19310779587037483,
 82: 0.1272136275533273,
 18: 0.09352214824198342,
 32: 0.09352214824198342,
 80: 0.09278348270013531,
 87: 0.07625212890548616,
 16: 0.06881320425489054,
 35: 0.06827348517633876,
 39: 0.05560302880540275,
 69: 0.05193860862632431,
 13: 0.044103800011901374,
 53: 0.04400190793497346,
 88: 0.02587500437863883,
 59: 0.025352737546870594,
 0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 14: 0.0,
 15: 0.0,
 17: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 33: 0.0,
 34: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,

In [49]:
def search_rank(word1,word2):

    if type(word1)==str:
        word1=WNlemma.lemmatize(word1)
        if word1 not in df.columns:
            temp1={}
        else:    
            temp1=df[word1].to_dict()
        
    else:
        temp1=word1
    if type(word2)==str: 
        word2=WNlemma.lemmatize(word2)
        if word2 in df.columns:    
            temp2=df[word2].to_dict()
        else:
            temp2={}
    else:
        temp2=word2
    if temp1 =={} and temp2=={}:
        return {}
    if temp2!={}:
        for key in temp1.keys():
            temp1[key]+=temp2[key]
    if temp1 =={}:
        res=dict()
        while len(temp2)>0:
            v = list(temp2.values())
            k = list(temp2.keys())
           
            res[k[v.index(max(v))]]=max(v)
            del temp2[k[v.index(max(v))]]
        return res
        
    res=dict()
    while len(temp1)>0:
        v = list(temp1.values())
        k = list(temp1.keys())
       
        res[k[v.index(max(v))]]=max(v)
        del temp1[k[v.index(max(v))]]
    return res

In [29]:
search_rank("series","game")

0.176220044906288
0.16972873847819767
0.07337846091604402
0.07153047829787565
0.03928816769422543
0.03727978230562749
0.02144009290022053
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


{38: 0.176220044906288,
 35: 0.16972873847819767,
 4: 0.07337846091604402,
 2: 0.07153047829787565,
 96: 0.03928816769422543,
 74: 0.03727978230562749,
 75: 0.02144009290022053,
 0: 0.0,
 1: 0.0,
 3: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 36: 0.0,
 37: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,


In [30]:
def general_search(query):
    temp=query.split()
    temp2=[]
    for word in temp:
        word=WNlemma.lemmatize(word)
        if word not in stop:
            temp2.append(word)
    res=search_rank1(temp2[0])
    for i in range(1,len(temp2)):
        res=search_rank(res,temp2[i])
    return res    

In [31]:
general_search("series and game")

0.176220044906288
0.16972873847819767
0.07337846091604402
0.07153047829787565
0.03928816769422543
0.03727978230562749
0.02144009290022053
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


{38: 0.176220044906288,
 35: 0.16972873847819767,
 4: 0.07337846091604402,
 2: 0.07153047829787565,
 96: 0.03928816769422543,
 74: 0.03727978230562749,
 75: 0.02144009290022053,
 0: 0.0,
 1: 0.0,
 3: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 36: 0.0,
 37: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,


In [50]:
class BTreeNode:
  def __init__(self, leaf=False):
    self.leaf = leaf
    self.keys = []
    self.child = []


# Tree
class BTree:
  def __init__(self, t):
    self.root = BTreeNode(True)
    self.t = t

    # Insert node
  def insert(self, k):
    root = self.root
    if len(root.keys) == (2 * self.t) - 1:
      temp = BTreeNode()
      self.root = temp
      temp.child.insert(0, root)
      self.split_child(temp, 0)
      self.insert_non_full(temp, k)
    else:
      self.insert_non_full(root, k)

    # Insert nonfull
  def insert_non_full(self, x, k):
    i = len(x.keys) - 1
    if x.leaf:
      x.keys.append(None)
      while i >= 0 and k < x.keys[i]:
        x.keys[i + 1] = x.keys[i]
        i -= 1
      x.keys[i + 1] = k
    else:
      while i >= 0 and k < x.keys[i]:
        i -= 1
      i += 1
      if len(x.child[i].keys) == (2 * self.t) - 1:
        self.split_child(x, i)
        if k > x.keys[i]:
          i += 1
      self.insert_non_full(x.child[i], k)

    # Split the child
  def split_child(self, x, i):
    t = self.t
    y = x.child[i]
    z = BTreeNode(y.leaf)
    x.child.insert(i + 1, z)
    x.keys.insert(i, y.keys[t - 1])
    z.keys = y.keys[t: (2 * t) - 1]
    y.keys = y.keys[0: t - 1]
    if not y.leaf:
      z.child = y.child[t: 2 * t]
      y.child = y.child[0: t - 1]

  # Print the tree
  def print_tree(self, x, l=0):
    print("Level ", l, " ", len(x.keys), end=":")
    for i in x.keys:
      print(i, end=" ")
    print()
    l += 1
    if len(x.child) > 0:
      for i in x.child:
        self.print_tree(i, l)

  # Search key in the tree
  def search_key(self, k, x=None,diff=10000,w=""):
       
    if x is not None:
      i = 0
      if not(any(char.isdigit() for char in x.keys[i])) or not(any(char.isdigit() for char in k)) or x.keys[i].isdigit():
          h=LD(wordSound(k),wordSound(x.keys[i]))
          if h<diff:
            diff=h
            w=x.keys[i]
      while i < len(x.keys) and k > x.keys[i]:
        i += 1
      if i < len(x.keys) and ( k == x.keys[i] ):
        return search_rank1(x.keys[i])
      elif x.leaf or i>=len(x.child):
        return self.search_key(w)
      else:
        k=WNlemma.lemmatize(k) 
        return self.search_key(k, x.child[i],diff,w)
      
    else:
      return self.search_key(k, self.root)

  def search_wild_card(self, k):
    res=[]
    temp=k.lower().strip().split("*")
    for i in df.columns:
        if(k[0]=="*" and len(temp)==1 and i[::-1].startswith(temp[1])):
            res.append(i)
        elif(k[-1]=="*" and len(temp)==1 and i.startswith(temp[0]) ):
              res.append(i)
        elif( i.startswith(temp[0]) and i[::-1].startswith(temp[-1])  ):
            temp1=temp[1:len(temp)-1]
           
            found=True
            for j in temp1:
                if j not in i:
                    found=False
            if found:
                res.append(i) 
     
    return self.search(" ".join(res)   ,True )             
        
  def search(self,query,result=False):
        
    temp=query.split()
    temp2=[]
    
    if "\"" in query:
        return literalSearch(query.replace("\"",""))
    
    for word in temp:
        if word not in stop:
            temp2.append(word)
    if len(temp2)==0:
        return literalSearch(" ".join(temp))
    elif "*" in temp2[0]:
        res=self.search_wild_card(temp2[0])
    else:    
        res=self.search_key(temp2[0])
    
    ress=[]
    for i in range(1,len(temp2)):
        if "*" in temp2[i]:
             
             res=search_rank(res,self.search_wild_card(temp2[i])) 
        else:        
            res=search_rank(res,self.search_key(temp2[i]))
   
    if result:
   
        return res
    for key,value in res.items():
        if value>0.0:
            ress.append(data[key]["artist"]+": "+data[key]["title"])
    return ress      

In [51]:
B = BTree(3)

for i in df.columns:
    B.insert(i)

B.print_tree(B.root)

print(B.search("amazing series"))

Level  0   2:fireplace permanent 
Level  1   2:business desperation 
Level  2   2:arrest bike 
Level  3   2:air ando 
Level  4   2:act age 
Level  5   2:30 acelera 
Level  6   2:1294 225 
Level  6   2:69 ability 
Level  5   2:addict aflowin 
Level  6   2:acting adam 
Level  6   2:adding adios 
Level  4   2:alive always 
Level  5   2:aisle alarm 
Level  6   2:airbag aircrash 
Level  6   2:al alabama 
Level  5   2:already although 
Level  6   2:alone along 
Level  6   2:alright also 
Level  3   2:back beauty 
Level  4   2:ate awaken 
Level  5   2:ask associate 
Level  6   2:ascending ash 
Level  6   2:asked asking 
Level  5   2:attracts avoid 
Level  6   2:attraction attractively 
Level  6   2:aught aunque 
Level  4   2:bald bark 
Level  5   2:bad bailando 
Level  6   2:backseat backwards 
Level  6   2:badada bag 
Level  5   2:band bank 
Level  6   2:balloon bambambambambambam 
Level  6   2:bandida bang 
Level  2   2:chill count 
Level  3   2:candle celebration 
Level  4   2:cabrón calif

In [52]:
B.search("a*")

['Radiohead: Karma Police',
 'Pink Floyd: Wish You Were Here',
 'Pink Floyd: Hey You',
 'Pink Floyd: Learning to Fly',
 'Radiohead: Subterranean Homesick Alien',
 'Tom Odell: Grow Old with Me',
 'Radiohead: No Surprises',
 'Keith Sweat: Open Invitation',
 'Keith Sweat: Knew It All Along',
 'Keith Sweat: Candy Store',
 'Tom Odell: Supposed to Be',
 'Radiohead: Palo Alto',
 'Radiohead: Climbing Up the Walls',
 "John Hiatt: 'Til I Get My Lovin' Back",
 "John Hiatt: Til I Get My Lovin' Back",
 'Radiohead: Airbag',
 'Radiohead: Electioneering',
 'Radiohead: Fitter Happier',
 'Pink Floyd: Another Brick in the Wall, Pt. 2',
 'The Beatles: Golden Slumbers',
 'Radiohead: Lucky',
 'The Beatles: Something',
 'John Hiatt: Adios to California',
 'Radiohead: Melatonin',
 'Keith Sweat: My Valentine',
 'John Hiatt: I Love That Girl',
 'Keith Sweat: Lady Dejour',
 "The Beatles: Maxwell's Silver Hammer",
 'The Beatles: The Long One',
 'Radiohead: The Tourist',
 'Ella Fitzgerald: The Tender Trap',
 'Pink

In [35]:
op=B.search_wild_card("a*")

0.08558293373066096
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.12557204456037405
0.08558293373066096
0.0641196635595109
0.04873966739791956
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.185405503459584
0.12557204456037405
0.08558293373066096
0.0641196635595109
0.

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.741622013838336
0.6278602228018703
0.32059831779755454
0.2436983369895978
0.13184136213162273
0.10340112794782481
0.10206926598953464
0.08888143333808404
0.08558293373066096
0.08462486328926375
0.044738368245034056
0.03724634178267037
0.02834694304479199
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7534322673622443
0.741622013838336
0.3847179813570655
0.2924380043875174
0.13184136213162273
0.10340112794782481
0.10206926598953464
0.08888143333808404
0.08558293373066096
0.08462486328926375
0.044738368245034056
0.03724634178267037
0.02834694304479199
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

0.0
0.0
0.0
1.1301484010433664
0.9270275172979201
0.6377914052698346
0.6350630525431226
0.4386570065812761
0.31020338384347446
0.2801437511872856
0.24178690474124803
0.13184136213162273
0.12967058859441635
0.1091171903907851
0.10214560595907571
0.10206926598953464
0.09613868047454242
0.08888143333808404
0.08558293373066096
0.08462486328926375
0.07834074614622225
0.07491453002535381
0.044738368245034056
0.03724634178267037
0.03007405197930272
0.02834694304479199
0.022910251022581627
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.1301484010433664
0.9270275172979201
0.6377914052698346
0.6350630525431226
0.4386570065812761
0.31020338384347446
0.2801437511872856
0.24178690474124803
0.179089051473809
0.15352715573101

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.2557204456037405
1.1124330207575042
0.6991827161026335
0.6720918214888989
0.6377914052698346
0.5337853357275925
0.5170056397391241
0.30630267902713626
0.29525340182945053
0.29024033667328614
0.2801437511872856
0.24178690474124803
0.2080934910371089
0.17116586746132192
0.15352715573101833
0.15174170927994518
0.12967058859441635
0.1091171903907851
0.10206926598953464
0.09352214824198342
0.09352214824198342
0.09278348270013531
0.08888143333808404
0.07877369043998966
0.07834074614622225
0.07491453002535381
0.07454317024017991
0.07061337262367288
0.06881320425489054
0.06827348517633876
0.06104386440849685
0.05193860862632431
0.051856089221313434
0.05052617177253487
0.044496295895093026
0.044103800011901374
0.04400190793497346
0.03724634178267037
0.03007405197930272
0.02834694304479199
0.022910251022581627
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

0.10206926598953464
0.09352214824198342
0.09352214824198342
0.09278348270013531
0.08888143333808404
0.07877369043998966
0.07834074614622225
0.07491453002535381
0.07454317024017991
0.07061337262367288
0.06881320425489054
0.06827348517633876
0.06104386440849685
0.05193860862632431
0.051856089221313434
0.05052617177253487
0.044496295895093026
0.044103800011901374
0.04400190793497346
0.03724634178267037
0.032432724301747554
0.03007405197930272
0.02834694304479199
0.022910251022581627
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0394605380554243
1.3812924901641146
0.7633023796621444
0.7208314888868185
0.6377914052698346
0.6204067676869489
0.5337853357275925
0.30630267902713626
0.29525340182945053
0.29024033667328614
0.2801437511872856
0.24178690474124803
0.2080934910371089
0.2037774503722

0.05193860862632431
0.051856089221313434
0.05052617177253487
0.044496295895093026
0.044103800011901374
0.04400190793497346
0.03724634178267037
0.032432724301747554
0.03007405197930272
0.02834694304479199
0.022910251022581627
0.015377917594599774
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0789210761108485
1.3812924901641146
0.7633023796621444
0.7208314888868185
0.6377914052698346
0.6204067676869489
0.5337853357275925
0.30630267902713626
0.30566617555835607
0.29525340182945053
0.29024033667328614
0.2801437511872856
0.24178690474124803
0.2080934910371089
0.17116586746132192
0.15352715573101833
0.15174170927994518
0.12967058859441635
0.1091171903907851
0.10206926598953464
0.09352214824198342
0.09352214824198342
0.09278348270013531
0.08888143333808404
0.07877369043998966
0.07834074614622225
0.07491453002535

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5.932976110706685
1.3812924901641146
0.7633023796621444
0.7208314888868185
0.6377914052698346
0.6204067676869489
0.5337853357275925
0.30630267902713626
0.30566617555835607
0.29525340182945053
0.29024033667328614
0.2801437511872856
0.24178690474124803
0.2080934910371089
0.17116586746132192
0.15352715573101833
0.15174170927994518
0.12967058859441635
0.1091171903907851
0.10206926598953464
0.09352214824198342
0.09352214824198342
0.09278348270013531
0.08987687958711676
0.08888143333808404
0.07877369043998966
0.07834074614622225
0.07491453002535381
0.07454317024017991
0.07061337262367288
0.06881320425489054
0.06827348517633876
0.06104386440849685
0.05193860862632431
0.051856089221313434
0.05052617177253487
0.044496295895093026
0.044103800011901374
0.04400190793497346
0.03724634178267037
0.032432724301747554
0.03007405197930272
0.02834

5.932976110706685
1.5273992154336908
0.8379074727054683
0.7208314888868185
0.6377914052698346
0.6204067676869489
0.5337853357275925
0.4401577507035725
0.3621237987023838
0.3481009535796778
0.30630267902713626
0.30566617555835607
0.29525340182945053
0.2801437511872856
0.2567488011919829
0.24178690474124803
0.2080934910371089
0.17852111243793442
0.16124946176927377
0.15352715573101833
0.12967058859441635
0.1091171903907851
0.10483216056264907
0.10206926598953464
0.09700627540843806
0.09352214824198342
0.09352214824198342
0.09278348270013531
0.08987687958711676
0.08615220526262127
0.07877369043998966
0.07834074614622225
0.07491453002535381
0.07454317024017991
0.07061337262367288
0.06881320425489054
0.06827348517633876
0.06104386440849685
0.058630206497374666
0.05193860862632431
0.05052617177253487
0.044496295895093026
0.044103800011901374
0.04400190793497346
0.03724634178267037
0.032432724301747554
0.03007405197930272
0.02834694304479199
0.022910251022581627
0.015377917594599774
0.0
0.0
0

1.5273992154336908
0.8725908141937089
0.7471955468768664
0.6684771672346187
0.6377914052698346
0.6324351116526573
0.6165029194083038
0.40855797272812816
0.38653850261834943
0.3621237987023838
0.3481009535796778
0.3469939130793134
0.31610092434545883
0.31610092434545883
0.3120282047561575
0.30566617555835607
0.2965356574917566
0.2801437511872856
0.26871199889566755
0.24178690474124803
0.22680742771997353
0.2080934910371089
0.18222692681122413
0.17852111243793442
0.1734430378601784
0.17299567426834722
0.16791596970741385
0.16772471683431614
0.16124946176927377
0.15352715573101833
0.15239439155008735
0.13888848913043775
0.12967058859441635
0.12375146426571218
0.12029620791721088
0.11053287123589448
0.10206926598953464
0.09352214824198342
0.09352214824198342
0.0861156131209936
0.08556624941743919
0.07937098046148859
0.07834074614622225
0.07454317024017991
0.07061337262367288
0.06881320425489054
0.06827348517633876
0.06104386440849685
0.059307954539193186
0.058630206497374666
0.051938608626

In [36]:
# from nltk.corpus import wordnet as wn
# from itertools import product

# wordx, wordy = "cat","dog"
# sem1, sem2 = wn.synsets(wordx), wn.synsets(wordy)

# maxscore = 0
# for i,j in list(product(*[sem1,sem2])):
 
#     score = i.wup_similarity(j)
#     if score!=None:
#         maxscore = score if maxscore < score else maxscore

In [37]:
# maxscore

In [38]:
def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list       

In [41]:
B.search("you b*")

0.20377745037223738
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.30566617555835607
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.40755490074447476
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.08356615827306163
0.08271758368071142
0.060772883010593845
0.059307954539193186
0.05739917460705796
0.05538811614164929
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044977773825722524
0.044808792119744006
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056
0.026364057990047936
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.018887251861187
0.39622607896675477
0.31610092434545883
0.31610092434545883
0.30038629735572814
0.26871199889566755
0.2029110143653724
0.19188829008101943
0.18222692681122413
0.1478968988919028
0.13410701822711935
0.13141114848045016
0.12517562962496684
0.11761970947193777
0.11053287123589448
0.10764542316219841
0.10225529370099189
0.10206926598953464
0.09727657760274312
0.0933113236768563
0.09115932451589076
0.

0.18222692681122413
0.1806662982438251
0.13410701822711935
0.13141114848045016
0.12812892367496398
0.12517562962496684
0.12302334075679819
0.11761970947193777
0.11053287123589448
0.10225728260627907
0.10225529370099189
0.10145525330185892
0.09727657760274312
0.0933113236768563
0.09115932451589076
0.08888143333808404
0.08556624941743919
0.08551460241302984
0.08356615827306163
0.08271758368071142
0.059307954539193186
0.05739917460705796
0.05538811614164929
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056
0.026364057990047936
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.324553427419543
0.5755752394703364
0.39622607896675477
0.31834595141317334
0.31610092434545883
0.31610092434545883
0.30038629735572814
0.289

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.7321083281640177
0.5956325508537155
0.5755752394703364
0.39622607896675477
0.31834595141317334
0.31610092434545883
0.31610092434545883
0.30038629735572814
0.26871199889566755
0.2654429053679305
0.2029110143653724
0.18222692681122413
0.1806662982438251
0.13410701822711935
0.13141114848045016
0.12812892367496398
0.12517562962496684
0.12302334075679819
0.11761970947193777
0.11053287123589448
0.10225728260627907
0.10225529370099189
0.10145525330185892
0.09727657760274312
0.0933113236768563
0.09115932451589076
0.08888143333808404
0.08556624941743919
0.08551460241302984
0.08356615827306163
0.08271758368071142
0.059307954539193186
0.05739917460705796
0.05538811614164929
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056
0.02834694304479199
0.026364057990047936
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

2.5472181296529683
0.5956325508537155
0.5755752394703364
0.39622607896675477
0.31834595141317334
0.31610092434545883
0.31610092434545883
0.30038629735572814
0.26871199889566755
0.2654429053679305
0.2029110143653724
0.18222692681122413
0.1806662982438251
0.13410701822711935
0.13141114848045016
0.12812892367496398
0.12517562962496684
0.12302334075679819
0.11761970947193777
0.11053287123589448
0.10225728260627907
0.10225529370099189
0.10145525330185892
0.09727657760274312
0.0933113236768563
0.09115932451589076
0.08888143333808404
0.08556624941743919
0.08551460241302984
0.08356615827306163
0.08271758368071142
0.059307954539193186
0.05739917460705796
0.05538811614164929
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056
0.02834694304479199
0.026364057990047936
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

...

0.08556624941743919
0.08551460241302984
0.08356615827306163
0.08271758368071142
0.059307954539193186
0.05739917460705796
0.05538811614164929
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056
0.02834694304479199
0.026364057990047936
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.743543114145033
0.9550378542395201
0.5956325508537155
0.5755752394703364
0.39622607896675477
0.31610092434545883
0.31610092434545883
0.30038629735572814
0.26871199889566755
0.2654429053679305
0.2029110143653724
0.18222692681122413
0.1806662982438251
0.13410701822711935
0.13141114848045016
0.12812892367496398
0.12517562962496684
0.12302334075679819
0.11761970947193777
0.11053287123589448
0.10225728260627907
0.10225529370099189
0.10145525330185

0.05538811614164929
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056
0.02834694304479199
0.026364057990047936
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
9.169985266750688
1.2733838056526936
0.5956325508537155
0.5755752394703364
0.39622607896675477
0.31610092434545883
0.31610092434545883
0.30038629735572814
0.26871199889566755
0.2654429053679305
0.2029110143653724
0.18222692681122413
0.1806662982438251
0.13410701822711935
0.13141114848045016
0.12812892367496398
0.12517562962496684
0.12302334075679819
0.11761970947193777
0.11053287123589448
0.10225728260627907
0.10225529370099189
0.10145525330185892
0.09727657760274312
0.0933113236768563
0.09115932451589076
0.09010848137994602
0.08888143333808404
0.0855662494174391

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
10.08698379342575
1.9100757084790405
0.5956325508537155
0.5755752394703364
0.39622607896675477
0.31610092434545883
0.31610092434545883
0.30038629735572814
0.26871199889566755
0.2654429053679305
0.2029110143653724
0.18222692681122413
0.1806662982438251
0.13410701822711935
0.13141114848045016
0.12812892367496398
0.12517562962496684
0.12302334075679819
0.11761970947193777
0.11053287123589448
0.10225728260627907
0.10225529370099189
0.10145525330185892
0.09727657760274312
0.0933113236768563
0.09115932451589076
0.09010848137994602
0.08888143333808404
0.08556624941743919
0.08551460241302984
0.08356615827306163
0.08271758368071142
0.059307954539193186
0.05739917460705796
0.05538811614164929
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056
0.02834694304479199
0.026364057990047936
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
11.003982320100812
2.5467676113053868
0.5956325508537155
0.5755752394703364
0.42961460524966
0.39622607896675477
0.31610092434545883
0.31610092434545883
0.26871199889566755
0.2654429053679305
0.20910928467118772
0.2029110143653724
0.2025127908397258
0.18222692681122413
0.15644627394366206
0.14426598045196443
0.13410701822711935
0.13141114848045016
0.12812892367496398
0.12517562962496684
0.12302334075679819
0.12062675743979963
0.12062675743979963
0.11761970947193777
0.11053287123589448
0.10225728260627907
0.10225529370099189
0.10145525330185892
0.09727657760274312
0.0933113236768563
0.09115932451589076
0.09010848137994602
0.08888143333808404
0.08556624941743919
0.08551460241302984
0.08356615827306163
0.08271758368071142
0.059307954539193186
0.05739917460705796
0.051740511249862814
0.05088709521482102
0.04807039954766982
0.044738368245034056
0.044315830094991765
0.04381896154822942
0.03536907252651513
0.03468334148824056

['Radiohead: Electioneering',
 "Keith Sweat: 'Til the Morning",
 'The Beatles: Goodbye',
 'Pink Floyd: The Fletcher Memorial Home',
 "The Beatles: Maxwell's Silver Hammer",
 'Keith Sweat: Candy Store',
 "The Beatles: I Want You (She's So Heavy)",
 "John Hiatt: 'Til I Get My Lovin' Back",
 "John Hiatt: Til I Get My Lovin' Back",
 'The Beatles: Golden Slumbers',
 'Radiohead: Fitter Happier',
 'The Beatles: The Long One',
 'Radiohead: Airbag',
 'Radiohead: Palo Alto',
 'Radiohead: Melatonin',
 'Pink Floyd: Eclipse',
 'Pink Floyd: Money',
 'The Beatles: The Ballad of John and Yoko',
 'The Beatles: You Never Give Me Your Money',
 'Pink Floyd: High Hopes',
 'John Hiatt: All the Way Under',
 'Zion & Lennox: Loco',
 'The Beatles: Mean Mr. Mustard',
 'The Beatles: Mean Mr Mustard',
 'The Beatles: Polythene Pam',
 'Radiohead: Pearly',
 'The Beatles: She Came in Through the Bathroom Window',
 'Keith Sweat: Knew It All Along',
 'Pink Floyd: Have a Cigar',
 'Tom Odell: Till I Lost',
 'Pink Floyd: C

In [40]:
B.search("\" you \" ")

['Pink Floyd: Wish You Were Here',
 'Pink Floyd: Eclipse',
 'Pink Floyd: Brain Damage',
 'Pink Floyd: Shine on You Crazy Diamond, Pts. 1-5',
 'Pink Floyd: The Fletcher Memorial Home',
 'Pink Floyd: Learning to Fly',
 'Pink Floyd: Comfortably Numb',
 'Pink Floyd: Money',
 "The Beatles: Maxwell's Silver Hammer",
 'The Beatles: The Long One',
 'The Beatles: Come and Get It',
 'The Beatles: The End',
 'The Beatles: Polythene Pam',
 'The Beatles: Because',
 'The Beatles: Something',
 'The Beatles: Come Together',
 'The Beatles: Old Brown Shoe',
 'The Beatles: The Ballad of John and Yoko',
 'The Beatles: You Never Give Me Your Money',
 "The Beatles: I Want You (She's So Heavy)",
 "The Beatles: Octopus's Garden",
 'The Beatles: Carry That Weight',
 'The Beatles: Golden Slumbers',
 'Pink Floyd: Time',
 'Pink Floyd: Have a Cigar',
 'Pink Floyd: Another Brick in the Wall, Pt. 2',
 'Pink Floyd: Hey You',
 'Radiohead: No Surprises',
 'Radiohead: How I Made My Millions',
 'Radiohead: Palo Alto',
 '